In [1]:
from datetime import datetime, date, time
import os 
import tarfile
import pandas as pd
import numpy as np
import seaborn as sns
import json
import requests
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
#P172: wikidata ID for ethnicity

WIKIDATA_QUERY = """
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    SELECT DISTINCT ?item WHERE {
      ?item p:P646 ?statement0.
      ?statement0 (ps:P646) "%s".
    }
    LIMIT 100
  }
}

"""

from wikidata.client import Client

client = Client()  # doctest: +SKIP
entity = client.get('Q49085', load=True)
entity.label

m'African Americans'

In [3]:
#extract tar data
data_folder_tar = './data/'
data_path_tar = os.path.join(data_folder_tar, 'MovieSummaries.tar.gz')


In [4]:
# open file
file = tarfile.open(data_path_tar)
  
# extract files
file.extractall('./data')

# close file
file.close()

In [5]:
Name = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']

In [6]:
#read different datas as dataframes
data_folder = './data/MovieSummaries'
data_folder_prime = './data/'
data_path = os.path.join(data_folder,'movie.metadata.tsv')
data_path_ratings = os.path.join(data_folder_prime,'title.ratings.tsv.gz')
data_path_name = os.path.join(data_folder_prime, 'title.basics.tsv.gz')
df_movie = pd.read_csv(data_path, sep ='\t', header = None, names = Name, na_values = ['{}', ' '], lineterminator='\n')
df_movies_kaggle = pd.read_csv(data_folder_prime + 'movies_metadata.csv')
df_movies_IMDb_ratings = pd.read_csv(data_path_ratings, sep ='\t', na_values = ['{}', ' '], lineterminator='\n')
df_movies_IMDb_name = pd.read_csv(data_path_name, sep ='\t', na_values = ['{}', ' '], lineterminator='\n')

C:\Users\Matthias Forestier\AppData\Local\Temp\ipykernel_15984\1682363459.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_kaggle = pd.read_csv(data_folder_prime + 'movies_metadata.csv')
C:\Users\Matthias Forestier\AppData\Local\Temp\ipykernel_15984\1682363459.py:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_IMDb_name = pd.read_csv(data_path_name, sep ='\t', na_values = ['{}', ' '], lineterminator='\n')


In [7]:
#character Metadata
Name_character = ['Wikipedia Movie ID','Freebase Movie ID', 'Movie release date', 'Character Name','Actor DOB', 'Actor gender', 'Actor height', 'Actor ethnicity', 'Actor Name', 'Actor age at movie release', 'unknown 1', 'unknown 2', 'Freebase character map']
data_path = os.path.join(data_folder,'character.metadata.tsv')
df_character = pd.read_csv(data_path, sep ='\t', header = None, names = Name_character, na_values = ' ', lineterminator='\n')

## First Analysis ##
On the primary test we wanted to see how many actors were linked to a movie. We discovered that all actors that played characters in the character database have a movie from the movie database linked to their character.

In [16]:
df_character = df_character.rename(columns={"Wikipedia Movie ID": "Wikipedia movie ID"})

In [17]:
df_character_movie = df_movie.merge(df_character, how ='inner', on=['Wikipedia movie ID'])
df_character_movie

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date_x,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Freebase Movie ID,...,Character Name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor Name,Actor age at movie release,unknown 1,unknown 2,Freebase character map
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",/m/03vyhn,...,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",/m/03vyhn,...,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",/m/03vyhn,...,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",/m/03vyhn,...,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",/m/03vyhn,...,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",/m/02w7zz8,...,NaN,NaN,NaN,NaN,NaN,Billy Morton,NaN,/m/0gchkcy,NaN,/m/0gc4lfm
450665,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",/m/02w7zz8,...,NaN,1982-01-28,NaN,NaN,NaN,Andrea Runge,19.0,/m/0gckh4f,NaN,/m/0gbx_rk
450666,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",/m/02w7zz8,...,NaN,NaN,F,NaN,NaN,Wendy Anderson,NaN,/m/0gcp8fv,NaN,/m/0gby01h
450667,12476867,/m/02w7zz8,Spliced,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0d060g"": ""Canada""}","{""/m/01jfsb"": ""Thriller"", ""/m/03npn"": ""Horror""...",/m/02w7zz8,...,NaN,NaN,NaN,NaN,NaN,Ariel Bastian,NaN,/m/0gdkb51,NaN,/m/0gdkb55


## Study of the Additional Dataset ##
In this stage we prepared a new dataframe by merging the two dataframes we imported from the IMDb database. The new dataframe we will obtain will have the average trating associated to the original title. Furthermore we changed some column names to facilitate future processing and merging.

In [18]:
df_IMDb_ex = df_movies_IMDb_name.merge(df_movies_IMDb_ratings, how ='inner', on=['tconst'])

In [19]:
df_IMDb_ex = df_IMDb_ex.rename(columns={"originalTitle": "Movie name"})
df_IMDb_ex

,tconst,titleType,primaryTitle,Movie name,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1922
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,259
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1735
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,174
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2548
...,...,...,...,...,...,...,...,...,...,...,...
1247714,tt9916690,tvEpisode,Horrid Henry Delivers the Milk,Horrid Henry Delivers the Milk,0,2012,\N,10,"Adventure,Animation,Comedy",7.4,6
1247715,tt9916720,short,The Nun 2,The Nun 2,0,2019,\N,10,"Comedy,Horror,Mystery",5.3,276
1247716,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,\N,8.0,8
1247717,tt9916766,tvEpisode,Episode #10.15,Episode #10.15,0,2019,\N,43,"Family,Game-Show,Reality-TV",6.7,21


## Creating and arranging the orignal datasets ##
We realised during our first study of the dataframes from the two original datasets that some informations were encoded (ethnicity), that some of dat wasn't expressed in the same format (the movie release date), and also  that some informations were difficult to reached: for example the movie genres, countries or languages. We decided to do some cleaning and simplifications on the data.

In [20]:
#Filtering
def filtering_function(column):
    _filtered = []
    i = 0
    for s in column:
        S_r = ''
        c = 1
        if (isinstance(s, str)):
            if s.find(',') == -1:
                S_r = s[(s.find(':')+3):(s.find(':}')-1)]
            else:
                a = s
                S_r = a[(a.find(':')+3):(a.find(',')-1)]
                a = a[(a.find(',')+1):]
                while c>0:
                    if a.find(',') == -1:
                        S_r =  S_r + ', ' + a[(a.find(':')+3):(a.find(':}')-1)]
                        c = 0
                    else:
                        S_r = S_r + ', ' + a[(a.find(':')+3):(a.find(',')-1)]
                        a = a[(a.find(',')+1):]
        else:
            S_r = 'None' 
        _filtered.append(S_r)
    return _filtered

#Language Filtering
language_filtered = filtering_function(df_movie['Movie languages'])
df_movie['Movie languages filtered'] = language_filtered

#Countries Filtering
countries_filtered = filtering_function(df_movie['Movie countries'])
df_movie['Movie countries filtered'] = countries_filtered

#Genres Filtering
genres_filtered = filtering_function(df_movie['Movie genres'])
df_movie['Movie genres filtered'] = genres_filtered

In [21]:
#movie release date to just year

def date(df_date):
    dates = []
    len_year = 4
    for date in df_date:
        if not (pd.isna(date)):
            date = str(date)
            if len(date) > len_year:
                dates.append(date[0:4])
            else:
                dates.append(date)
        else:
            dates.append(date)
            
    return dates

In [22]:
df_movie["startYear"] = date(df_movie["Movie release date"])

In [23]:
df_movie

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Movie languages filtered,Movie countries filtered,Movie genres filtered,startYear
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",2001
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",English Language,United States of America,"Mystery, Biographical film, Drama, Crime Drama",2000
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",Norwegian Language,Norway,"Crime Fiction, Drama",1988
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",English Language,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",1987
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",German Language,Germany,Drama,1983
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",English Language,United States of America,Drama,2011
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",English Language,"Ireland, United Kingdom","Biographical film, Drama, Documentary",2011
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",English Language,United States of America,"Satire, Comedy",1972
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",Japanese Language,Japan,"Science Fiction, Japanese Movies, Adventure, A...",1992


## Studying the complementarity of the additional and original datasets ##
In the next step we merge the two datasets to see how many movies from the original datasets have actually a rating in the additional dataset. We will then see how many actors from the original datasets have ratings.

In [27]:
df_exploitable = df_movie.merge(df_IMDb_ex, how ='inner', on=['Movie name', 'startYear'])

In [28]:
df_exploitable = df_exploitable.drop_duplicates(subset = "Wikipedia movie ID") 

In [29]:
df_exploitable

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Movie languages filtered,...,startYear,tconst,titleType,primaryTitle,isAdult,endYear,runtimeMinutes,genres,averageRating,numVotes
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,...,2001,tt0228333,movie,Ghosts of Mars,0,\N,98,"Action,Horror,Sci-Fi",4.9,55271
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",Norwegian Language,...,1988,tt0094806,movie,Brun bitter,0,\N,83,"Crime,Drama",5.7,40
3,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...","Silent film, English Language",...,1913,tt0002894,short,The Gangsters,0,\N,35,"Comedy,Short",6.8,16
4,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",English Language,...,1938,tt0029852,movie,Alexander's Ragtime Band,0,\N,106,"Drama,Music,Musical",6.9,2160
5,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/01g6gs"": ""Black-an...",English Language,...,1930,tt0021335,movie,Sarah and Son,0,\N,86,"Drama,Romance",5.4,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46506,15394941,/m/03m6zh4,Gopi Kishan,1994-12-02,NaN,NaN,"{""/m/03k50"": ""Hindi Language""}","{""/m/03rk0"": ""India""}","{""/m/07s9rl0"": ""Drama"", ""/m/02kdv5l"": ""Action""...",Hindi Language,...,1994,tt0109922,movie,Gopi Kishan,0,\N,161,"Action,Comedy,Drama",5.8,913
46507,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",English Language,...,1941,tt0033660,movie,The Ghost Train,0,\N,85,"Comedy,Horror",6.2,1531
46508,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",English Language,...,2011,tt1816585,tvMovie,Mermaids: The Body Found,0,\N,82,Sci-Fi,4.6,1656
46509,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",English Language,...,2011,tt1606259,movie,Knuckle,0,\N,97,"Biography,Documentary,Drama",6.8,3081


Here we will transform the encoded ethnicity IDs into readable natural language.

In [45]:
def get_ethnicity(freebase_id):
    # freebase -> wikidata
    query = WIKIDATA_QUERY % freebase_id
    req = requests.get("https://query.wikidata.org/sparql",
                       params={"format": "json", "query": query})
    try:
        data = req.json()
        wikidata_id = data["results"]["bindings"][0]["itemLabel"]["value"]
    except:
        return np.nan
    
    client = Client()
    entity = client.get(wikidata_id, load=True)
    
    return entity.label
    

In [46]:
temp_df = df_character['Actor ethnicity'].groupby(df_character['Actor ethnicity']).agg(['count'])
temp_df = temp_df.reset_index()

In [47]:
temp_temp_df = temp_df.copy()
et = []
pb = []
for i in range(len(temp_df)):
    df_temp_ethnicity = temp_df.iloc[i:i+1]
    for ethnicity in df_temp_ethnicity['Actor ethnicity'] :
        eth_tra = get_ethnicity(ethnicity)
        j = 0
        while ((type(eth_tra) == float) & (j<100)):
            #print(type(get_ethnicity(ethnicity)))
            eth_tra = get_ethnicity(ethnicity)
            j += 1
            if j == 100:
                pb.append(i)
        et.append(eth_tra)
#temp_temp_df['Actor ethnicity'] = et

In [48]:
print(get_ethnicity(temp_df['Actor ethnicity'][29]))

nan


Some of the encoded ethnicity didn't have any linked articles in wikipedia as the one present line 29 of temp_df.

## Obtaining our final Dataframe ##
In the next step we obtain our final dataframe, the one we will work with. To work we need four main feature and we will maybe add some more later but for the moment it will be: Movie name, Actor name, averageRating (the average rating), and finally the startYear.

In [40]:
df_exploitable_character = df_exploitable.merge(df_character, how ='inner', on=['Wikipedia movie ID'])
df_years_ascending = df_exploitable_character.sort_values(['startYear'], axis=0, ascending=True, inplace=False, kind='quicksort')

In [41]:
df_years_ascending

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date_x,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Movie languages filtered,...,Character Name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor Name,Actor age at movie release,unknown 1,unknown 2,Freebase character map
212680,1082508,/m/044ggd,Roundhay Garden Scene,1888,NaN,0.03,"{""/m/06ppq"": ""Silent film""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",Silent film,...,NaN,NaN,F,NaN,NaN,Sarah Whitley,NaN,/m/0k2pzt,NaN,/m/0k2pz0
212679,1082508,/m/044ggd,Roundhay Garden Scene,1888,NaN,0.03,"{""/m/06ppq"": ""Silent film""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",Silent film,...,NaN,NaN,M,NaN,NaN,Adolphe Le Prince,NaN,/m/0k2pzn,NaN,/m/0k2pyv
212681,1082508,/m/044ggd,Roundhay Garden Scene,1888,NaN,0.03,"{""/m/06ppq"": ""Silent film""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",Silent film,...,NaN,NaN,M,NaN,NaN,Joseph Whitley,NaN,/m/0k2pzz,NaN,/m/0k2pz6
212682,1082508,/m/044ggd,Roundhay Garden Scene,1888,NaN,0.03,"{""/m/06ppq"": ""Silent film""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",Silent film,...,NaN,NaN,F,NaN,NaN,Harriet Hartley,NaN,/m/0k2p_3,NaN,/m/0k2pzd
96169,644824,/m/02_1qn,Dickson Greeting,1891,NaN,0.05,NaN,"{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/0219x_"": ""Indie""}",None,...,NaN,1860-08-03,M,NaN,/m/03lmx1,William Dickson,NaN,/m/0jx879,NaN,/m/081ks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201924,36127934,/m/0k0tq97,Maleficent,2014-03-14,NaN,NaN,NaN,NaN,"{""/m/01hmnh"": ""Fantasy""}",None,...,NaN,1975-06-04,F,1.73,/m/027n1m6,Angelina Jolie,38.0,/m/0k3d7ym,NaN,/m/0f4vbz
201926,36127934,/m/0k0tq97,Maleficent,2014-03-14,NaN,NaN,NaN,NaN,"{""/m/01hmnh"": ""Fantasy""}",None,...,NaN,1956-01-09,F,1.52,/m/02w7gg,Imelda Staunton,58.0,/m/0k3d7_m,NaN,/m/01tspc6
201927,36127934,/m/0k0tq97,Maleficent,2014-03-14,NaN,NaN,NaN,NaN,"{""/m/01hmnh"": ""Fantasy""}",None,...,NaN,1958-03-03,F,1.65,/m/02w7gg,Miranda Richardson,56.0,/m/0k3d7_s,NaN,/m/0525b
128957,36534974,/m/0kdy118,Noah,2014-03-28,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/01hmnh"": ""Fantasy""}",English Language,...,Ila,1990-04-15,F,1.68,/m/0d7wh,Emma Watson,23.0,/m/0kdy2xy,/m/0kdy2v8,/m/0134w7


In [42]:
df_exploitable_character

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date_x,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Movie languages filtered,...,Character Name,Actor DOB,Actor gender,Actor height,Actor ethnicity,Actor Name,Actor age at movie release,unknown 1,unknown 2,Freebase character map
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,...,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,...,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,...,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,...,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",English Language,...,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300180,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",English Language,...,NaN,1900-01-14,M,NaN,NaN,Wilfrid Lawson,41.0,/m/0gcgrfv,NaN,/m/025w40r
300181,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",English Language,...,NaN,1891-08-08,M,NaN,NaN,Morland Graham,NaN,/m/0gckksz,NaN,/m/0gc08qy
300182,23851782,/m/06_vb43,The Ghost Train,1941-05-03,NaN,82.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",English Language,...,NaN,1869,M,NaN,NaN,D.J. Williams,NaN,/m/0gdk36r,NaN,/m/0gcwx9k
300183,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",English Language,...,NaN,1938-11-26,M,NaN,NaN,Rich Little,33.0,/m/02vbk4r,NaN,/m/03m6t5


In [43]:
df_final = df_exploitable_character[['Actor Name','Movie name','startYear','averageRating']]
df_final

,Actor Name,Movie name,startYear,averageRating
0,Wanda De Jesus,Ghosts of Mars,2001,4.9
1,Natasha Henstridge,Ghosts of Mars,2001,4.9
2,Ice Cube,Ghosts of Mars,2001,4.9
3,Jason Statham,Ghosts of Mars,2001,4.9
4,Clea DuVall,Ghosts of Mars,2001,4.9
...,...,...,...,...
300180,Wilfrid Lawson,The Ghost Train,1941,6.2
300181,Morland Graham,The Ghost Train,1941,6.2
300182,D.J. Williams,The Ghost Train,1941,6.2
300183,Rich Little,Another Nice Mess,1972,5.8


In [44]:
df_final_visualization = df_final['Actor Name'].groupby(df_final['Actor Name']).agg(['count'])
df_final_visualization[df_final_visualization['count']>30]

,count
Actor Name,
A.K Hangal,31
Abhishek Bachchan,40
Achala Sachdev,56
Adam Baldwin,31
Adam Sandler,33
...,...
Woody Allen,37
Woody Harrelson,56
Xander Berkeley,44


## Bonus ##
In the next cells we tried looking at an other dataset to evaluate its potential but sadly there wasn't enough data for us to use it. We also looked on the ratings and discovered that there we also coming from the IMDb database so it was just redundant.

In [30]:
df_movies_kaggle.drop(df_movies_kaggle[df_movies_kaggle['vote_average'] == 0.0].index, inplace = True)
df_movies_kaggle = df_movies_kaggle.rename(columns={"original_title": "Movie name", "imdb_id": "tconst"})
df_movies_test = df_movies_kaggle.merge(df_IMDb_ex, how ='inner', on=['Movie name', 'tconst'])

In [31]:
df_movies_test

,adult,belongs_to_collection,budget,genres_x,homepage,id,tconst,original_language,Movie name,overview,...,vote_count,titleType,primaryTitle,isAdult,startYear,endYear,runtimeMinutes,genres_y,averageRating,numVotes
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,5415.0,movie,Toy Story,0,1995,\N,81,"Adventure,Animation,Comedy",8.3,989276
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,2413.0,movie,Jumanji,0,1995,\N,104,"Adventure,Comedy,Family",7.0,346227
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,92.0,movie,Grumpier Old Men,0,1995,\N,101,"Comedy,Romance",6.6,27936
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,34.0,movie,Waiting to Exhale,0,1995,\N,124,"Comedy,Drama,Romance",5.9,11148
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,173.0,movie,Father of the Bride Part II,0,1995,\N,106,"Comedy,Family,Romance",6.0,38844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34704,False,NaN,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",NaN,390959,tt0265736,en,Shadow of the Blair Witch,"In this true-crime documentary, we delve into ...",...,2.0,tvMovie,Shadow of the Blair Witch,0,2000,\N,45,"Horror,Mystery",5.2,487
34705,False,NaN,0,"[{'id': 27, 'name': 'Horror'}]",NaN,289923,tt0252966,en,The Burkittsville 7,A film archivist revisits the story of Rustin ...,...,1.0,tvMovie,The Burkittsville 7,0,2000,\N,30,Horror,6.4,265
34706,False,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,222848,tt0112613,en,Caged Heat 3000,It's the year 3000 AD. The world's most danger...,...,1.0,movie,Caged Heat 3000,0,1995,\N,80,"Action,Sci-Fi",3.6,387
34707,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",...,26.0,movie,Robin Hood,0,1991,\N,104,"Action,Adventure,Drama",5.7,5312


In [32]:
df_movies_test_prime = df_movies_test.merge(df_movie, how ='inner', on=['Movie name'])

In [33]:
df_movies_test_prime

,adult,belongs_to_collection,budget,genres_x,homepage,id,tconst,original_language,Movie name,overview,...,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Movie languages filtered,Movie countries filtered,Movie genres filtered,startYear_y
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-11-19,361958736.0,77.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0556j8"": ""Buddy film"", ""/m/03k9fj"": ""Adve...",English Language,United States of America,"Buddy film, Adventure, Children's/Family, Comp...",1995
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/0hj3n2k"": ""Fanta...","French Language, English Language","United States of America, Japan","Thriller, Fantasy Adventure, Japanese Movies, ...",1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,71518503.0,101.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0556j8"": ""...","Italian Language, English Language",United States of America,"Romantic comedy, Buddy film, Comedy",1995
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n0w"": ""Ensemble Film"", ""/m/06w2n3t"": ...",English Language,United States of America,"Ensemble Film, Female buddy film, Comedy-drama...",1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-12-08,76594107.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06qm3"": ""Screwball comedy"", ""/m/02l7c8"": ...",English Language,United States of America,"Screwball comedy, Romance Film, Family Film, C...",1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27525,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,84710,tt0036975,en,Jungle Woman,Paula the ape woman (Acquanetta) is alive and ...,...,1944-06-01,NaN,61.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""H...",English Language,United States of America,"Science Fiction, Horror, Sci-Fi Horror, Jungle...",1944
27526,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,39562,tt0889600,en,To Be Fat Like Me,"Pretty, popular, and slim high-schooler Aly Sc...",...,2007-01-08,NaN,89.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/015w9s"": ""Televisi...",English Language,"United States of America, Canada","Drama, Television movie",2007
27527,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,14008,tt0294425,en,Cadet Kelly,Hyperactive teenager Kelly is enrolled into a ...,...,2002-03-08,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/0hqxf"": ""Family Film"", ""/m/0bj8m2"": ""Chil...",English Language,"Unit

In [34]:
df_tot = df_exploitable.merge(df_movies_test_prime, how ='outer', on=['Movie name'])
df_tot = df_tot.drop_duplicates(subset = "Movie name") 